# The power of SQLAlchemy

David Mcilwee
- david@mcilwee.me

### What we're going to cover
* SQL recap
* Tables, Metadata & Modelling
* Exercise 1: A tree
* Sessions, events and mapper extensions
* Exercise 2: Audit extension

### What you will need
* python 2.7 / 3.4 (This notebook runs 3.4)
* pip install sqlalchemy jupyter

In [1]:
# Boring functions
from datetime import datetime
from pprint import pprint
from operator import itemgetter

def printq(q):
    print(str(q)+"\n")
    

### SQLAlchemy setup
Let's create an engine (a DB connection)

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:')

And our MetaData, to help manage our view of the data model

In [3]:
from sqlalchemy import MetaData
metadata = MetaData()
metadata

MetaData(bind=None)

Our example Domain, films and actors

In [4]:
from sqlalchemy import Table, ForeignKey, Column, DateTime, Unicode, Integer

films = Table('films', metadata,
    Column('id', Integer, primary_key=True),
    Column('title', Unicode(40), nullable=False),
    Column('date_produced', DateTime),
    Column('director_id', Integer, ForeignKey('directors.id'), nullable=True)
)

actors = Table('actors', metadata,
  Column('id', Integer, primary_key=True),
  Column('name', Unicode, nullable=False)
)

actors = Table('directors', metadata,
  Column('id', Integer, primary_key=True),
  Column('name', Unicode, nullable=False)
)

actors_to_films = Table('actors_to_films', metadata,
  Column('actor_id', Integer, ForeignKey('actors.id')),
  Column('film_id', Integer, ForeignKey('films.id'))
)

genres = Table('genre', metadata,
  Column('id', Integer, primary_key=True),
  Column('name', Unicode, nullable=False)
)

genre_to_films = Table('genre_to_films', metadata,
  Column('genre_id', Integer, ForeignKey('genre.id')),
  Column('film_id', Integer, ForeignKey('films.id'))
)

Lets create our tables

In [5]:
metadata.create_all(bind=engine)

### Inserting

Lets fill our DB it with some example data

In [6]:
engine.execute(
    films.insert(),
    title='Star Wars Episode IV - A New Hope',
    date_produced=datetime(1977, 3, 26)
)

Lets do a bulk insert

In [7]:
movie_data = [
    {'title': 'The Thing', 'date_produced': datetime.now(), 'kind': 'Action'},
    {'title': 'The Godfather', 'date_produced': datetime(1972, 1, 1), 'kind': 'Crime, Drama'},
    {'title': 'The Shawshank Redemption', 'date_produced': datetime(1994, 1, 1), 'kind': 'Crime, Drama'},
    {'title': 'Schindlers List', 'date_produced': datetime(1993, 1, 1), 'kind': 'Biography, Drama, History'},
    {'title': 'Raging Bull', 'date_produced': datetime(1980, 1, 1), 'kind': 'Biography, Drama, Sport'},
    {'title': 'Casablanca', 'date_produced': datetime(1942, 1, 1), 'kind': ' Drama, Romance, War'},
    {'title': 'Citizen Kane', 'date_produced': datetime(1941, 1, 1), 'kind': 'Drama, Mystery'},
    {'title': 'Gone with the Wind', 'date_produced': datetime(1939, 1, 1), 'kind': 'Drama, History, Romance'},
    {'title': 'The Wizard of Oz', 'date_produced': datetime(1939, 1, 1), 'kind': 'Adventure, Family, Fantasy'},
    {'title': 'One Flew Over the Cuckoos Nest', 'date_produced': datetime(1975, 1, 1), 'kind': 'Drama'},
    {'title': 'Lawrence of Arabia', 'date_produced': datetime(1962, 1, 1), 'kind': 'Adventure, Biography, Drama'},
    {'title': 'Vertigo', 'date_produced': datetime(1958, 1, 1), 'kind': 'Mystery, Romance, Thriller'},
    {'title': 'Psycho', 'date_produced': datetime(1960, 1, 1), 'kind': ' Horror, Mystery, Thriller'},
    {'title': 'The Godfather: Part II', 'date_produced': datetime(1974, 1, 1), 'kind': 'Crime, Drama'},
    {'title': 'On the Waterfront', 'date_produced': datetime(1954, 1, 1), 'kind': 'Crime, Drama, Thriller'},
    {'title': 'Sunset Boulevard', 'date_produced': datetime(1950, 1, 1), 'kind': ' Drama, Film-Noir'},
    {'title': 'Forrest Gump', 'date_produced': datetime(1994, 1, 1), 'kind': ' Comedy, Drama'},
    {'title': 'The Sound of Music', 'date_produced': datetime(1965, 1, 1), 'kind': 'Biography, Drama, Family'},
    {'title': '12 Angry Men', 'date_produced': datetime(1957, 1, 1), 'kind': 'Crime, Drama'},
    {'title': 'West Side Story', 'date_produced': datetime(1961, 1, 1), 'kind': 'Crime, Drama, Musical'},    
]

# Strip out the 'kind' column, we're going to add that later
insert_data = [{'title': r['title'], 'date_produced': r['date_produced']} for r in movie_data]
               
engine.execute(films.insert(), insert_data)

Lets check the bulk insert SQL to make sure it's actually doing it the way we want to.

In [8]:
query = films.insert(insert_data, bind=engine)
printq(query)

INSERT INTO films (title, date_produced) VALUES (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?), (?, ?)



Notice here our usage of ``bind``, this is to tell SQLAlchemy what we're going to be inserting in against so it can do the right thing. Normally this would happen automatically in ``engine.execute``.

## Querying, our bread and butter

Just a plain select, by default SQLAlchemy will query all columns defined

In [9]:
results = engine.execute(films.select())
for row in results:
    pprint(row)

(1, 'Star Wars Episode IV - A New Hope', datetime.datetime(1977, 3, 26, 0, 0), None)
(2, 'The Thing', datetime.datetime(2016, 10, 27, 21, 7, 19, 393845), None)
(3, 'The Godfather', datetime.datetime(1972, 1, 1, 0, 0), None)
(4, 'The Shawshank Redemption', datetime.datetime(1994, 1, 1, 0, 0), None)
(5, 'Schindlers List', datetime.datetime(1993, 1, 1, 0, 0), None)
(6, 'Raging Bull', datetime.datetime(1980, 1, 1, 0, 0), None)
(7, 'Casablanca', datetime.datetime(1942, 1, 1, 0, 0), None)
(8, 'Citizen Kane', datetime.datetime(1941, 1, 1, 0, 0), None)
(9, 'Gone with the Wind', datetime.datetime(1939, 1, 1, 0, 0), None)
(10, 'The Wizard of Oz', datetime.datetime(1939, 1, 1, 0, 0), None)
(11, 'One Flew Over the Cuckoos Nest', datetime.datetime(1975, 1, 1, 0, 0), None)
(12, 'Lawrence of Arabia', datetime.datetime(1962, 1, 1, 0, 0), None)
(13, 'Vertigo', datetime.datetime(1958, 1, 1, 0, 0), None)
(14, 'Psycho', datetime.datetime(1960, 1, 1, 0, 0), None)
(15, 'The Godfather: Part II', datetime.dat

### Actually specifying the columns we care about.

Both ``films.select`` and ``select([films])`` are equivalent, in some cases one is easier to use over the other.

In [10]:
from sqlalchemy import select
query = select([films.c.id, films.c.title])

printq(query)

SELECT films.id, films.title 
FROM films



What happens if I only want the first row / value?

In [11]:
row = engine.execute(films.select()).fetchone()
print(row)

(1, 'Star Wars Episode IV - A New Hope', datetime.datetime(1977, 3, 26, 0, 0), None)


In [12]:
first_id = engine.execute(films.select()).scalar()
print(first_id)

1


### Filtering

Pretty much like SQL!

In [13]:
query = films.select().where(films.c.id > 5)
printq(query)

SELECT films.id, films.title, films.date_produced, films.director_id 
FROM films 
WHERE films.id > :id_1



In [14]:
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

(6, 'Raging Bull', datetime.datetime(1980, 1, 1, 0, 0), None)
(7, 'Casablanca', datetime.datetime(1942, 1, 1, 0, 0), None)
(8, 'Citizen Kane', datetime.datetime(1941, 1, 1, 0, 0), None)
(9, 'Gone with the Wind', datetime.datetime(1939, 1, 1, 0, 0), None)
(10, 'The Wizard of Oz', datetime.datetime(1939, 1, 1, 0, 0), None)
(11, 'One Flew Over the Cuckoos Nest', datetime.datetime(1975, 1, 1, 0, 0), None)
(12, 'Lawrence of Arabia', datetime.datetime(1962, 1, 1, 0, 0), None)
(13, 'Vertigo', datetime.datetime(1958, 1, 1, 0, 0), None)
(14, 'Psycho', datetime.datetime(1960, 1, 1, 0, 0), None)
(15, 'The Godfather: Part II', datetime.datetime(1974, 1, 1, 0, 0), None)
(16, 'On the Waterfront', datetime.datetime(1954, 1, 1, 0, 0), None)
(17, 'Sunset Boulevard', datetime.datetime(1950, 1, 1, 0, 0), None)
(18, 'Forrest Gump', datetime.datetime(1994, 1, 1, 0, 0), None)
(19, 'The Sound of Music', datetime.datetime(1965, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None)

Ask the audience for some queryies.


## Ordering / Limit

Limit / offsets are just another call

In [15]:
query = select([
    films.c.title,
    films.c.date_produced
]).limit(4)
printq(query)

rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films.title, films.date_produced 
FROM films
 LIMIT :param_1

('Star Wars Episode IV - A New Hope', datetime.datetime(1977, 3, 26, 0, 0))
('The Thing', datetime.datetime(2016, 10, 27, 21, 7, 19, 393845))
('The Godfather', datetime.datetime(1972, 1, 1, 0, 0))
('The Shawshank Redemption', datetime.datetime(1994, 1, 1, 0, 0))


In [16]:
query = select([
    films.c.title,
    films.c.date_produced
]).limit(2).offset(1)
printq(query)

rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films.title, films.date_produced 
FROM films
 LIMIT :param_1 OFFSET :param_2

('The Thing', datetime.datetime(2016, 10, 27, 21, 7, 19, 393845))
('The Godfather', datetime.datetime(1972, 1, 1, 0, 0))


Ordering can be done similarly

In [17]:
from sqlalchemy import func
query = select([
    films.c.title,
    films.c.date_produced
]).limit(2).order_by(films.c.date_produced)
printq(query)

rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films.title, films.date_produced 
FROM films ORDER BY films.date_produced
 LIMIT :param_1

('Gone with the Wind', datetime.datetime(1939, 1, 1, 0, 0))
('The Wizard of Oz', datetime.datetime(1939, 1, 1, 0, 0))


Or ordering explicitly ascending/descending

In [18]:
from sqlalchemy import func
query = select([
    films.c.title,
    films.c.date_produced
]).limit(2).order_by(films.c.date_produced.desc())
printq(query)

rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films.title, films.date_produced 
FROM films ORDER BY films.date_produced DESC
 LIMIT :param_1

('The Thing', datetime.datetime(2016, 10, 27, 21, 7, 19, 393845))
('The Shawshank Redemption', datetime.datetime(1994, 1, 1, 0, 0))


### Inserting in some more query data
How we would typically do a many to many relationship insert. Keep this in mind later on as the declarative ORM makes this **vastly** easier.

In [19]:
# Inserting in the genre mapping

# STOP AND THINK OF THE DIFFERENCE BETWEEN TABLES AND A NESTED DATA STRUCTURE / TREES

unique_genres = set( r.strip() for r in movie_data for r in r['kind'].split(','))
engine.execute(genres.insert(), [{'name': r} for r in unique_genres])

all_movies_lookup = dict(
    (row.title, row.id)
    for row in 
    engine.execute(films.select())
)
all_genres_lookup = dict(
    (row.name, row.id)
    for row in 
    engine.execute(genres.select())
)

to_insert_data = []
for row in movie_data:
    for row_genre in map(str.strip, row['kind'].split(',')):    
        to_insert_data.append(
            {
                'film_id': all_movies_lookup[row['title']],
                'genre_id': all_genres_lookup[row_genre],
            }
        )
    
# Insert our Many to Many mapping
engine.execute(genre_to_films.delete(), to_insert_data)
engine.execute(genre_to_films.insert(), to_insert_data)


## SQL joins
As easy as SQL, if not a bit easier!

In [20]:
query = select([
    genres.c.name,
    films.c.title
]).select_from(
    genres.join(genre_to_films, genres.c.id==genre_to_films.c.genre_id
    ).join(films, films.c.id==genre_to_films.c.film_id)
).order_by(genres.c.name).limit(10)
printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT genre.name, films.title 
FROM genre JOIN genre_to_films ON genre.id = genre_to_films.genre_id JOIN films ON films.id = genre_to_films.film_id ORDER BY genre.name
 LIMIT :param_1

('Action', 'The Thing')
('Adventure', 'The Wizard of Oz')
('Adventure', 'Lawrence of Arabia')
('Biography', 'Schindlers List')
('Biography', 'Raging Bull')
('Biography', 'Lawrence of Arabia')
('Biography', 'The Sound of Music')
('Comedy', 'Forrest Gump')
('Crime', 'The Godfather')
('Crime', 'The Shawshank Redemption')


### Aliasing
Sometimes you will come across situations where you need to join to the same table twice.

Here I want to find out for every film, what films are older than it?

In [21]:
film_a, film_b = films.alias(), films.alias()
query = select([
    film_a, film_b
]).where(
    film_a.c.date_produced > film_b.c.date_produced
).order_by(film_a.c.title)  # using "name" here would be ambiguous

printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films_1.id, films_1.title, films_1.date_produced, films_1.director_id, films_2.id, films_2.title, films_2.date_produced, films_2.director_id 
FROM films AS films_1, films AS films_2 
WHERE films_1.date_produced > films_2.date_produced ORDER BY films_1.title

(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 7, 'Casablanca', datetime.datetime(1942, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 8, 'Citizen Kane', datetime.datetime(1941, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 9, 'Gone with the Wind', datetime.datetime(1939, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 10, 'The Wizard of Oz', datetime.datetime(1939, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 16, 'On the Waterfront', datetime.datetime(1954, 1, 1, 0, 0), None)
(20, '12 Angry Men', datetime.datetime(1957, 1, 1, 0, 0), None, 17, 'Sunset Boulevard', d

## Aggregates (grouping)
Once again very much like SQL. Here we're going to use a SQLite specific function ``group_concat`` to do string aggregation.
``sqlalchemy.func`` allows us to use any database function.

In [22]:
from sqlalchemy import func
query = select([
    genres.c.name,
    func.group_concat(films.c.title, ', ')
]).select_from(
    genres.join(genre_to_films, genres.c.id==genre_to_films.c.genre_id
    ).join(films, films.c.id==genre_to_films.c.film_id)
).group_by(films.c.title)
printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT genre.name, group_concat(films.title, :group_concat_2) AS group_concat_1 
FROM genre JOIN genre_to_films ON genre.id = genre_to_films.genre_id JOIN films ON films.id = genre_to_films.film_id GROUP BY films.title

('Drama', '12 Angry Men, 12 Angry Men')
('War', 'Casablanca, Casablanca, Casablanca')
('Mystery', 'Citizen Kane, Citizen Kane')
('Drama', 'Forrest Gump, Forrest Gump')
('Romance', 'Gone with the Wind, Gone with the Wind, Gone with the Wind')
('Drama', 'Lawrence of Arabia, Lawrence of Arabia, Lawrence of Arabia')
('Thriller', 'On the Waterfront, On the Waterfront, On the Waterfront')
('Drama', 'One Flew Over the Cuckoos Nest')
('Thriller', 'Psycho, Psycho, Psycho')
('Sport', 'Raging Bull, Raging Bull, Raging Bull')
('History', 'Schindlers List, Schindlers List, Schindlers List')
('Film-Noir', 'Sunset Boulevard, Sunset Boulevard')
('Drama', 'The Godfather, The Godfather')
('Drama', 'The Godfather: Part II, The Godfather: Part II')
('Drama', 'The Shawshank Redemption, The 

## Raw SQL
Sometimes you just have to write your own.

In [23]:
from sqlalchemy import text
query = select([
    genres.c.name,
    text('group_concat(films.title, "|SEP|") AS MyCol')
]).select_from(
    genres.join(genre_to_films, genres.c.id==genre_to_films.c.genre_id
    ).join(films, films.c.id==genre_to_films.c.film_id)
).group_by(films.c.title)
printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT genre.name, group_concat(films.title, "|SEP|") AS MyCol 
FROM genre JOIN genre_to_films ON genre.id = genre_to_films.genre_id JOIN films ON films.id = genre_to_films.film_id GROUP BY films.title

('Drama', '12 Angry Men|SEP|12 Angry Men')
('War', 'Casablanca|SEP|Casablanca|SEP|Casablanca')
('Mystery', 'Citizen Kane|SEP|Citizen Kane')
('Drama', 'Forrest Gump|SEP|Forrest Gump')
('Romance', 'Gone with the Wind|SEP|Gone with the Wind|SEP|Gone with the Wind')
('Drama', 'Lawrence of Arabia|SEP|Lawrence of Arabia|SEP|Lawrence of Arabia')
('Thriller', 'On the Waterfront|SEP|On the Waterfront|SEP|On the Waterfront')
('Drama', 'One Flew Over the Cuckoos Nest')
('Thriller', 'Psycho|SEP|Psycho|SEP|Psycho')
('Sport', 'Raging Bull|SEP|Raging Bull|SEP|Raging Bull')
('History', 'Schindlers List|SEP|Schindlers List|SEP|Schindlers List')
('Film-Noir', 'Sunset Boulevard|SEP|Sunset Boulevard')
('Drama', 'The Godfather|SEP|The Godfather')
('Drama', 'The Godfather: Part II|SEP|The Godfather: Part II'

You can also embed it in any of the other clauses

In [24]:
from sqlalchemy import text
query = select([
    genres.c.name,
    text('group_concat(films.title, "|SEP|") AS MyCol')
]).select_from(
    genres.join(genre_to_films, genres.c.id==genre_to_films.c.genre_id
    ).join(films, films.c.id==genre_to_films.c.film_id)
).where(
    text('films.title LIKE "%W%"')
).group_by(films.c.title)
printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT genre.name, group_concat(films.title, "|SEP|") AS MyCol 
FROM genre JOIN genre_to_films ON genre.id = genre_to_films.genre_id JOIN films ON films.id = genre_to_films.film_id 
WHERE films.title LIKE "%W%" GROUP BY films.title

('Romance', 'Gone with the Wind|SEP|Gone with the Wind|SEP|Gone with the Wind')
('Drama', 'Lawrence of Arabia|SEP|Lawrence of Arabia|SEP|Lawrence of Arabia')
('Thriller', 'On the Waterfront|SEP|On the Waterfront|SEP|On the Waterfront')
('Drama', 'One Flew Over the Cuckoos Nest')
('Drama', 'The Shawshank Redemption|SEP|The Shawshank Redemption')
('Fantasy', 'The Wizard of Oz|SEP|The Wizard of Oz|SEP|The Wizard of Oz')
('Musical', 'West Side Story|SEP|West Side Story|SEP|West Side Story')


### Sub-queries
Sometimes you need to compose your query using multiple building blocks. Subqueries allow you to isolate a portion of your selectable.

What have I used this for?
* more performant queries (incredebly subject-dependent)
* composing results via ``UNION`` / ``UNION ALL``
* filtering against a complex domain -> reducing complexity 

But be warned! It will have an effect on the Query Plan of your operation.

In [25]:
from sqlalchemy import func, subquery

genre_subq = select([
        genre_to_films.c.film_id, 
        func.count(genres.c.id).label('genre_count')
]).select_from(
    genre_to_films.join(
    genres, genres.c.id==genre_to_films.c.genre_id
)).alias('my_subq')


query = select([
    films.c.title,
    genre_subq.c.genre_count
]).select_from(
    films.outerjoin(
        genre_subq, films.c.id==genre_subq.c.film_id
    )
)

printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

SELECT films.title, my_subq.genre_count 
FROM films LEFT OUTER JOIN (SELECT genre_to_films.film_id AS film_id, count(genre.id) AS genre_count 
FROM genre_to_films JOIN genre ON genre.id = genre_to_films.genre_id) AS my_subq ON films.id = my_subq.film_id

('Star Wars Episode IV - A New Hope', None)
('The Thing', None)
('The Godfather', None)
('The Shawshank Redemption', None)
('Schindlers List', None)
('Raging Bull', None)
('Casablanca', None)
('Citizen Kane', None)
('Gone with the Wind', None)
('The Wizard of Oz', None)
('One Flew Over the Cuckoos Nest', None)
('Lawrence of Arabia', None)
('Vertigo', None)
('Psycho', None)
('The Godfather: Part II', None)
('On the Waterfront', None)
('Sunset Boulevard', None)
('Forrest Gump', None)
('The Sound of Music', None)
('12 Angry Men', None)
('West Side Story', 49)


### CTE's

Common table expressions (a much neater way of composing queries)

In [26]:
from sqlalchemy import func, subquery

genre_subq = select([
        genre_to_films.c.film_id, 
        func.count(genres.c.id).label('genre_count')
]).select_from(
    genre_to_films.join(
    genres, genres.c.id==genre_to_films.c.genre_id
)).cte('my_subq')


query = select([
    films.c.title,
    genre_subq.c.genre_count
]).select_from(
    films.outerjoin(
        genre_subq, films.c.id==genre_subq.c.film_id
    )
)

printq(query)
rows = engine.execute(query).fetchall()
for r in rows:
    pprint(r)

WITH my_subq AS 
(SELECT genre_to_films.film_id AS film_id, count(genre.id) AS genre_count 
FROM genre_to_films JOIN genre ON genre.id = genre_to_films.genre_id)
 SELECT films.title, my_subq.genre_count 
FROM films LEFT OUTER JOIN my_subq ON films.id = my_subq.film_id

('Star Wars Episode IV - A New Hope', None)
('The Thing', None)
('The Godfather', None)
('The Shawshank Redemption', None)
('Schindlers List', None)
('Raging Bull', None)
('Casablanca', None)
('Citizen Kane', None)
('Gone with the Wind', None)
('The Wizard of Oz', None)
('One Flew Over the Cuckoos Nest', None)
('Lawrence of Arabia', None)
('Vertigo', None)
('Psycho', None)
('The Godfather: Part II', None)
('On the Waterfront', None)
('Sunset Boulevard', None)
('Forrest Gump', None)
('The Sound of Music', None)
('12 Angry Men', None)
('West Side Story', 49)


# Exercise 1: Binary trees

A binary search tree is a rooted binary tree, whose internal nodes each store a key (and optionally, an associated value) and each have two distinguished sub-trees, commonly denoted left and right. The tree additionally satisfies the binary search tree property, which states that the key in each node must be greater than all keys stored in the left sub-tree, and less than all keys in the right sub-tree.

https://en.wikipedia.org/wiki/Binary_search_tree


Let's try and model the following tree
```
 python
 pythonesque
 pythoness
 pythonic
 pythons

 pytho -> nes -> que
              -> s
       -> n   -> ic
              -> s 
```

Tasks
-----

1. Create a mapping for our Domain Model
2. Insert the test data
3. Query and print out the entire tree
4. Implement a search query

In [27]:
bnode = Table('node', metadata,
    Column('id', Integer, primary_key=True),
    Column('value', Unicode, nullable=False),
    Column('left_id', Integer, ForeignKey('node.id'), nullable=True),
    Column('right_id', Integer, ForeignKey('node.id'), nullable=True),    
    extend_existing=True
)

# I'm going to cheat and save a lot of work here
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
Base = declarative_base(metadata=metadata)
from sqlalchemy.orm import sessionmaker
# create a configured "Session" class
Session = sessionmaker(bind=engine)
# create a Session
session = Session()

class BNode(Base):
    """ Node in a binary tree """
    __table__ = bnode
    
    left = relationship('BNode', remote_side=[bnode.c.id], foreign_keys=[bnode.c.left_id])
    
    right = relationship('BNode', remote_side=[bnode.c.id], foreign_keys=[bnode.c.right_id])
    

Base.metadata.create_all(engine)    
btree_session = Session()


In [28]:
# Inserting some test data
# python
# pythonesque
# pythoness
# pythonic
# pythons
#
# pytho -> nes -> que
#              -> s
#       -> n -> ic
#            -> s 

root_node = BNode(value='pytho')
# 1st level
root_node.left = BNode(value='n')
root_node.right = BNode(value='nes')
# 2nd level 
root_node.left.left = BNode(value='s')
root_node.left.right = BNode(value='ic')
root_node.right.left = BNode(value='s')
root_node.right.right = BNode(value='que')

In [29]:
btree_session.add(root_node)
btree_session.commit()

pprint(session.query(BNode.id, BNode.left_id, BNode.right_id, BNode.value).all())

[(1, None, None, 's'),
 (2, None, None, 'ic'),
 (3, None, None, 's'),
 (4, None, None, 'que'),
 (5, 1, 2, 'n'),
 (6, 3, 4, 'nes'),
 (7, 5, 6, 'pytho')]


In [30]:
def print_tree_node(level, node):
    child_nodes = node.left and node.right
    print("\t"*level, node.value, " -> " if child_nodes else "")
    if node.left:
        print_tree_node(level+1, node.left)
    if node.right:
        print_tree_node(level+1, node.right)

print_tree_node(0, root_node)

 pytho  -> 
	 n  -> 
		 s 
		 ic 
	 nes  -> 
		 s 
		 que 


Lets make a recursive query to print out the value of each full value.

In [31]:
from sqlalchemy import and_, or_, literal, case

filter_node_q = session.query(
    BNode.id,
    BNode.left_id,
    BNode.right_id,
    BNode.value,
    literal(None).label('left_match')
).filter(
    BNode.value.ilike('py%')  # We will find all nodes (including this) down/up the tree
).cte(recursive=True)


filter_node_q_alias = filter_node_q.alias()

In [32]:
from sqlalchemy.orm import aliased

bnode_alias = aliased(BNode)

query_down = filter_node_q.union_all(
    select([
        bnode_alias.id.label('child_id'),
        bnode_alias.left_id,
        bnode_alias.right_id,
        bnode_alias.value,
        case([
            (bnode_alias.id==filter_node_q_alias.c.left_id, True)
        ], else_=False).label('left_match')
    ]).where(
        # Traverse DOWN the tree
        or_(
            bnode_alias.id==filter_node_q_alias.c.left_id,
            bnode_alias.id==filter_node_q_alias.c.right_id,
        )
        
    )
)

pprint(session.query(query_down).all())
pprint(session.query(
    query_down.c.id, 
    func.group_concat(query_down.c.value, '')
        
).all())
    

[(7, 5, 6, 'pytho', None),
 (5, 1, 2, 'n', 1),
 (6, 3, 4, 'nes', 0),
 (1, None, None, 's', 1),
 (2, None, None, 'ic', 0),
 (3, None, None, 's', 1),
 (4, None, None, 'que', 0)]
[(4, 'pythonnessicsque')]


## Mapping your domain 

SQLAlchemy has 2 core parts

* Expressions API
* ORM

Expression API is the lowest level, gives us the base to stand on.

Gives us ``Table`` and queries 

ORM allows us to work with Classes/Objects in a much neater way.

Provides the ``Mapper`` and ``Mapping``


(Mappers are there to help us and generally keep out of the way. But it's good to know they exist/what they are.)



### Our film/genre mapping (Take 2)

In [33]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base(metadata=metadata)

In [34]:
class Actor(Base):
    """ An actor """
    __tablename__ = 'actors'
    __table_args__ = {'extend_existing': True}

    id = Column('id', Integer, primary_key=True)
    name = Column('name', Unicode, nullable=False)
   

class Genre(Base):
    """ A Genre """
    __table__ = genres  # Re-using our existing table
    
    
class Director(Base):
    """ A Director """
    __tablename__ = 'directors'
    __table_args__ = {'extend_existing': True}

    id = Column('id', Integer, primary_key=True)
    name = Column('name', Unicode, nullable=False)


In [35]:
Base.metadata.create_all(engine)

### Defining relationships
Just mapping tables to classes does not give us a whole lot unless we can actually define some deeper relations

In [36]:
from sqlalchemy.orm import relationship
from sqlalchemy import PrimaryKeyConstraint

actors_to_films = Table('actors_to_films', metadata,
    Column('actor_id', Integer, ForeignKey('actors.id'), primary_key=True),
    Column('film_id', Integer, ForeignKey('films.id'), primary_key=True),
    extend_existing=True
)

genre_to_films = Table('genre_to_films', metadata,
    Column('genre_id', Integer, ForeignKey('genre.id')),
    Column('film_id', Integer, ForeignKey('films.id')),
    PrimaryKeyConstraint('genre_id', 'film_id'),
    extend_existing=True
                       
)


class Film(Base):
    """ A film """
    __tablename__ = 'films'
    __table_args__ = {'extend_existing': True}
    
    id = Column('id', Integer, primary_key=True)
    title = Column('title', Unicode(40), nullable=False)
    date_produced = Column('date_produced', DateTime)
    
    # One to Many
    director_id = Column('director_id', Integer, ForeignKey("directors.id"), nullable=True)
    director = relationship("Director")
    
    # Many to many relationships
    actors = relationship("Actor", secondary=actors_to_films)
    
    genres = relationship("Genre", secondary=genre_to_films)


## Querying with the ORM, sessions
SQLAlchemy uses the ``Unit of Work`` pattern to handle manipulating the ORM. So we need to take a quick dive into how that works.

A ``session`` is a holding ground for all queries/changes made to the objects mapped by the ORM. It maintains it's own internal state to track changes, and will (generally) automatically resolve dependent changes for you.

## Take a quick detour to
http://docs.sqlalchemy.org/en/latest/orm/session_basics.html and http://www.aosabook.org/en/sqlalchemy.html#fig.sqlalchemy.session

In [37]:
from sqlalchemy.orm import sessionmaker


# create a configured "Session" class
Session = sessionmaker(bind=engine)

# create a Session
session = Session()

In [38]:
session.is_active

True

In [39]:
res = session.query(Film)
for obj in res:
    pprint(obj)

In [40]:
res = session.query(Film).filter(Film.title.ilike('%a%'))
for obj in res:
    pprint((obj.id, obj.title))

(1, 'Star Wars Episode IV - A New Hope')
(3, 'The Godfather')
(4, 'The Shawshank Redemption')
(6, 'Raging Bull')
(7, 'Casablanca')
(8, 'Citizen Kane')
(10, 'The Wizard of Oz')
(12, 'Lawrence of Arabia')
(15, 'The Godfather: Part II')
(16, 'On the Waterfront')
(17, 'Sunset Boulevard')
(20, '12 Angry Men')


All of the querying work we did with ``Table`` above applies to the ORM! The only major difference is referencing columns by their property name ``Film.id`` vs their column ``films.c.id``.

In [41]:
session.close()  # Closing out our session

### Hybrid properties

Sometimes you've a property that just doesn't quite match directly to SQL. You can write some mapping code to do this via Python, and then some hints to enable it directly in SQL!

In [42]:
from sqlalchemy.ext.hybrid import hybrid_property

class GenreToFilm(Base):
    __table__ = genre_to_films
    film = relationship("Film")

class Film2(Film):
    """ A film """
    __table__ = Film.__table__
    __table_args__ = {'extend_existing': True}
    
    director = relationship("Director")
    
    # Many to many relationships
    actors = relationship("Actor", secondary=actors_to_films)
    
    genres = relationship("Genre", secondary=genre_to_films)
    
    @hybrid_property
    def csv_genres(self):
        return ", ".join([g.name for g in self.genres])
    
    @csv_genres.expression
    def csv_genres(cls):
        return select(
            [func.group_concat(Genre.name, ', ')]
        ).select_from(
            Genre.__table__.join(
                genre_to_films, genre_to_films.c.genre_id==Genre.id
            )
        ).where(genre_to_films.c.film_id==cls.id
        ).label('csv_genres')
    
   

In [43]:
# Emitting a new query every time
for f in session.query(Film2):
    print((f.title, f.csv_genres))
    
print('-'*40)

# Pushing all the work to SQL
for m in session.query(Film2.title, Film2.csv_genres):
    print(m)


('Star Wars Episode IV - A New Hope', '')
('The Thing', 'Action')
('The Godfather', 'Crime, Drama')
('The Shawshank Redemption', 'Crime, Drama')
('Schindlers List', 'Biography, Drama, History')
('Raging Bull', 'Biography, Drama, Sport')
('Casablanca', 'Drama, Romance, War')
('Citizen Kane', 'Drama, Mystery')
('Gone with the Wind', 'Drama, History, Romance')
('The Wizard of Oz', 'Adventure, Family, Fantasy')
('One Flew Over the Cuckoos Nest', 'Drama')
('Lawrence of Arabia', 'Adventure, Biography, Drama')
('Vertigo', 'Mystery, Romance, Thriller')
('Psycho', 'Horror, Mystery, Thriller')
('The Godfather: Part II', 'Crime, Drama')
('On the Waterfront', 'Crime, Drama, Thriller')
('Sunset Boulevard', 'Drama, Film-Noir')
('Forrest Gump', 'Comedy, Drama')
('The Sound of Music', 'Biography, Drama, Family')
('12 Angry Men', 'Crime, Drama')
('West Side Story', 'Crime, Drama, Musical')
----------------------------------------
('Star Wars Episode IV - A New Hope', None)
('The Thing', 'Action')
('The

In [44]:
# The executed SQL query
print(session.query(Film2.title, Film2.csv_genres))

SELECT films.title AS films_title, (SELECT group_concat(genre.name, ?) AS group_concat_1 
FROM genre JOIN genre_to_films ON genre_to_films.genre_id = genre.id 
WHERE genre_to_films.film_id = films.id) AS csv_genres 
FROM films


## Joinedload / query optimization
Being able to query relationships on the fly is quite powerful, but let's see what the ORM is doing under the hood to make this all happen.

In [45]:
# Enable echo=True on engine
for f in session.query(Film2):
    print((f.title, f.genres))

('Star Wars Episode IV - A New Hope', [])
('The Thing', [<__main__.Genre object at 0x7f5271312dd8>])
('The Godfather', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('The Shawshank Redemption', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('Schindlers List', [<__main__.Genre object at 0x7f527136bcf8>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527136be10>])
('Raging Bull', [<__main__.Genre object at 0x7f527136bcf8>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f5271312eb8>])
('Casablanca', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f5271312908>, <__main__.Genre object at 0x7f5271312f28>])
('Citizen Kane', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f5271312a20>])
('Gone with the Wind', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527136be10>, <__main__.Genre o

but SQLAlchemy always has a way. We can controll this via loading strategies

http://docs.sqlalchemy.org/en/latest/orm/loading_relationships.html?highlight=joinedload#relationship-loading-techniques

In [46]:
from sqlalchemy.orm import joinedload, subqueryload

for f in session.query(Film2).options(joinedload('genres')):
    print((f.title, f.genres))

('Star Wars Episode IV - A New Hope', [])
('The Thing', [<__main__.Genre object at 0x7f527131f6a0>])
('The Godfather', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('The Shawshank Redemption', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('Schindlers List', [<__main__.Genre object at 0x7f527131f908>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131f978>])
('Raging Bull', [<__main__.Genre object at 0x7f527131fa90>, <__main__.Genre object at 0x7f527131f908>, <__main__.Genre object at 0x7f527138a9e8>])
('Casablanca', [<__main__.Genre object at 0x7f527138a588>, <__main__.Genre object at 0x7f527138a240>, <__main__.Genre object at 0x7f527138a9e8>])
('Citizen Kane', [<__main__.Genre object at 0x7f527138a518>, <__main__.Genre object at 0x7f527138a9e8>])
('Gone with the Wind', [<__main__.Genre object at 0x7f527138a588>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre o

In [47]:
for f in session.query(Film2).options(subqueryload('genres')):
    print((f.title, f.genres))

('Star Wars Episode IV - A New Hope', [])
('The Thing', [<__main__.Genre object at 0x7f527131f4e0>])
('The Godfather', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('The Shawshank Redemption', [<__main__.Genre object at 0x7f527138a940>, <__main__.Genre object at 0x7f527138a9e8>])
('Schindlers List', [<__main__.Genre object at 0x7f527131fb70>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131fc18>])
('Raging Bull', [<__main__.Genre object at 0x7f527131fb70>, <__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131fcc0>])
('Casablanca', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131fd68>, <__main__.Genre object at 0x7f527131fdd8>])
('Citizen Kane', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131feb8>])
('Gone with the Wind', [<__main__.Genre object at 0x7f527138a9e8>, <__main__.Genre object at 0x7f527131fc18>, <__main__.Genre o

## ORM Events 

Sometimes you want to do something just before / after a Mapping is committed.

In [48]:
from sqlalchemy.event import listen

def no_stairway_films(target, value, oldvalue, initiator):
    if 'Waynes World' in value:
        raise ValueError('No Stairway!')
        
listen(Film.title, 'set', no_stairway_films)

In [49]:
session.add(Film(title='Waynes World 2', ))
try:
    session.commit()
except ValueError as e:
    print(e)

ValueError: No Stairway!

So what events are possible to hook in to?

In [ ]:
from sqlalchemy import events

dir(events)

In [ ]:
from sqlalchemy.orm import events
dir(events)

help(events.QueryEvents)

# Example 2: Access Log
Take the example of auditing access to a particular resource. It can be quite daunting to individually instrument every place you make a mutation. What happens if you miss one?

Or perhaps extending your Domain model with additional audit-specific logic just isn't working for you

In [50]:
from sqlalchemy import event
from sqlalchemy.orm import Query

@event.listens_for(Query, "before_compile", retval=True)
def check_access_to_genres(query):
    for desc in query.column_descriptions:
        if desc['type'] is Genre:
            print("Accessing Genre! {}".format(desc))

In [51]:
for r in session.query(Genre):
    print(r.name)

Accessing Genre! {'name': 'Genre', 'aliased': False, 'type': <class '__main__.Genre'>, 'entity': <class '__main__.Genre'>, 'expr': <class '__main__.Genre'>}
Sport
Family
Comedy
Crime
Romance
Thriller
Film-Noir
War
Horror
Biography
Mystery
Action
Drama
History
Adventure
Fantasy
Musical


Success!

In [52]:
for r in session.query(Genre.name):
    print(r.name)

Sport
Family
Comedy
Crime
Romance
Thriller
Film-Noir
War
Horror
Biography
Mystery
Action
Drama
History
Adventure
Fantasy
Musical


Not quite successful, let's look at this in a bit more detail.

In [54]:
@event.listens_for(Query, "before_compile", retval=True)
def check_access_to_genres(query):
    for desc in query.column_descriptions:
        print("{}".format(desc))
        
for r in session.query(Genre.name):
    print(r.name)

{'name': 'name', 'aliased': False, 'type': Unicode(), 'entity': <class '__main__.Genre'>, 'expr': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x7f527139aa98>}
{'name': 'name', 'aliased': False, 'type': Unicode(), 'entity': <class '__main__.Genre'>, 'expr': <sqlalchemy.orm.attributes.InstrumentedAttribute object at 0x7f527139aa98>}
Sport
Family
Comedy
Crime
Romance
Thriller
Film-Noir
War
Horror
Biography
Mystery
Action
Drama
History
Adventure
Fantasy
Musical


# The end!